<a href="https://colab.research.google.com/github/2023ad05120-lab/AI_VolumeNet_Capstone_Project/blob/main/generate_synthetic_dataset_coco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# generate_synthetic_dataset.py
# Generates a synthetic dataset of regular-shaped household items (images, masks, metadata, COCO JSON)
# Usage: python generate_synthetic_dataset.py --out_dir /path/to/out --num 200

import os
import json
import csv
import random
import math
from PIL import Image, ImageDraw, ImageFilter, ImageFont
import numpy as np
import argparse
import zipfile

def draw_box(draw, bbox, color):
    draw.rectangle(bbox, fill=color)

def draw_cylinder(draw, bbox, color):
    x0,y0,x1,y1 = bbox
    draw.rectangle([x0, y0+ (y1-y0)*0.15, x1, y1-(y1-y0)*0.05], fill=color)
    draw.ellipse([x0, y0, x1, y0 + (y1-y0)*0.3], fill=color)
    draw.ellipse([x0, y1-(y1-y0)*0.15, x1, y1], fill=color)

def draw_sphere(draw, bbox, color):
    draw.ellipse(bbox, fill=color)

def draw_bottle(draw, bbox, color):
    x0,y0,x1,y1 = bbox
    w = x1-x0; h = y1-y0
    neck_w = w*0.4
    nx0 = x0 + (w-neck_w)/2
    nx1 = nx0 + neck_w
    draw.rectangle([x0, y0+0.25*h, x1, y1], fill=color)
    draw.rectangle([nx0, y0, nx1, y0+0.25*h], fill=color)
    draw.ellipse([x0, y1-0.08*h, x1, y1], fill=color)

def draw_packet(draw, bbox, color):
    draw.rounded_rectangle(bbox, radius=8, fill=color)

shape_draw_fns = {
    "box": draw_box,
    "cylinder": draw_cylinder,
    "sphere": draw_sphere,
    "can": draw_cylinder,
    "bottle": draw_bottle,
    "packet": draw_packet
}

def bbox_from_mask_np(m):
    ys, xs = np.where(m)
    if len(xs)==0:
        return [0,0,0,0]
    x0,y0 = int(xs.min()), int(ys.min())
    x1,y1 = int(xs.max()), int(ys.max())
    return [x0,y0,x1,y1]

def main(out_dir, num_images, W=640, H=480):
    os.makedirs(out_dir, exist_ok=True)
    img_dir = os.path.join(out_dir, "images")
    mask_dir = os.path.join(out_dir, "masks")
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)
    meta_csv = os.path.join(out_dir, "metadata.csv")
    coco_json = os.path.join(out_dir, "annotations_coco.json")
    zip_path = os.path.join(out_dir, "synthetic_dataset.zip")

    random.seed(42)
    np.random.seed(42)
    # reference (credit card) in cm
    ref_w_cm = 8.56
    ref_h_cm = 5.398

    categories = [
        {"id":1, "name":"box"},
        {"id":2, "name":"cylinder"},
        {"id":3, "name":"sphere"},
        {"id":4, "name":"can"},
        {"id":5, "name":"bottle"},
        {"id":6, "name":"packet"},
        {"id":7, "name":"reference"}
    ]

    coco = {"info":{"description":"AIVolumeNet synthetic dataset"},"images":[], "annotations":[], "categories":categories}
    metadata_rows = []

    ann_id = 1
    img_id = 1

    for i in range(num_images):
        shape = random.choice(list(shape_draw_fns.keys()))
        # assign real-world dims (cm)
        if shape in ["box","packet"]:
            L = round(random.uniform(6,25), 2)
            W_cm = round(random.uniform(4,20), 2)
            H_cm = round(random.uniform(2,20), 2)
            real_dims = {"L":L,"W":W_cm,"H":H_cm}
        elif shape in ["cylinder","can","bottle"]:
            diameter = round(random.uniform(4,12), 2)
            height = round(random.uniform(8,25), 2)
            real_dims = {"D":diameter,"H":height}
        elif shape=="sphere":
            diameter = round(random.uniform(4,20), 2)
            real_dims = {"D":diameter}
        else:
            real_dims = {}

        bg_color = tuple(np.random.randint(200,255,size=3).tolist())
        img = Image.new("RGB", (W,H), bg_color)
        mask = Image.new("L", (W,H), 0)
        draw = ImageDraw.Draw(img)
        mdraw = ImageDraw.Draw(mask)

        # pixels per cm simulating distance variation
        ppcm = random.uniform(6.5,12.0)
        if shape in ["box","packet"]:
            pw = int(real_dims["L"] * ppcm)
            ph = int(real_dims["H"] * ppcm)
        elif shape in ["cylinder","can","bottle"]:
            pw = int(real_dims["D"] * ppcm)
            ph = int(real_dims["H"] * ppcm)
        elif shape=="sphere":
            pw = int(real_dims["D"] * ppcm)
            ph = int(real_dims["D"] * ppcm)
        else:
            pw = ph = 40

        margin = 20
        pw = max(20, min(pw, W-2*margin))
        ph = max(20, min(ph, H-2*margin-40))
        x0 = random.randint(margin, W - pw - margin)
        y0 = random.randint(margin, H - ph - margin - 40)
        x1 = x0 + pw
        y1 = y0 + ph

        color = tuple(np.random.randint(40,200,size=3).tolist())
        # draw object and mask (object mask value 255)
        shape_draw_fns[shape](draw, (x0,y0,x1,y1), color)
        shape_draw_fns[shape](mdraw, (x0,y0,x1,y1), 255)

        # soft blur shadow
        img = img.filter(ImageFilter.GaussianBlur(radius=1))
        # draw reference credit-card at bottom-right area (mask value 128)
        ref_pixel_w = int(ref_w_cm * ppcm)
        ref_pixel_h = int(ref_h_cm * ppcm)
        rx1 = W - random.randint(24,40)
        rx0 = rx1 - ref_pixel_w
        ry1 = H - random.randint(18,30)
        ry0 = ry1 - ref_pixel_h
        if rx0 < 5: rx0 = 5; rx1 = rx0 + ref_pixel_w
        if ry0 < 5: ry0 = 5; ry1 = ry0 + ref_pixel_h
        draw.rectangle([rx0,ry0,rx1,ry1], fill=(80,80,80))
        mdraw.rectangle([rx0,ry0,rx1,ry1], fill=128)
        try:
            font = ImageFont.load_default()
            draw.text((rx0+3, ry0+3), "REF", fill=(255,255,255), font=font)
        except:
            pass

        img_path = os.path.join(img_dir, f"img_{i:04d}.png")
        mask_path = os.path.join(mask_dir, f"mask_{i:04d}.png")
        img.save(img_path, optimize=True)
        mask.save(mask_path, optimize=True)

        mask_np = np.array(mask)
        obj_mask = (mask_np==255).astype(np.uint8)
        ref_mask = (mask_np==128).astype(np.uint8)
        obj_bbox = bbox_from_mask_np(obj_mask)
        ref_bbox = bbox_from_mask_np(ref_mask)
        ref_pix_w = max(1, ref_bbox[2]-ref_bbox[0])
        est_ppcm = ref_pix_w / ref_w_cm if ref_pix_w>0 else None

        if shape in ["box","packet"]:
            volume_cm3 = real_dims["L"] * real_dims["W"] * real_dims["H"]
        elif shape in ["cylinder","can","bottle"]:
            r = real_dims["D"]/2.0
            volume_cm3 = math.pi * r*r * real_dims["H"]
        elif shape=="sphere":
            r = real_dims["D"]/2.0
            volume_cm3 = 4.0/3.0 * math.pi * r**3
        else:
            volume_cm3 = None

        metadata_row = {
            "image_id": os.path.basename(img_path),
            "shape": shape,
            "real_dims": json.dumps(real_dims),
            "proj_bbox_px": json.dumps(obj_bbox),
            "ref_bbox_px": json.dumps(ref_bbox),
            "ppcm_used_for_render": round(ppcm,3),
            "est_ppcm_from_ref": round(est_ppcm,3) if est_ppcm else None,
            "volume_cm3": round(volume_cm3,3) if volume_cm3 else None
        }
        metadata_rows.append(metadata_row)

        # COCO entries
        coco["images"].append({"id": img_id, "file_name": os.path.basename(img_path), "width":W, "height":H})
        cat_map = {"box":1,"cylinder":2,"sphere":3,"can":4,"bottle":5,"packet":6,"reference":7}
        x,y,x2,y2 = obj_bbox
        coco["annotations"].append({
            "id": ann_id, "image_id": img_id, "category_id": cat_map[shape],
            "bbox": [x,y,x2-x,y2-y], "area": (x2-x)*(y2-y), "iscrowd":0
        })
        ann_id += 1
        coco["annotations"].append({
            "id": ann_id, "image_id": img_id, "category_id": cat_map["reference"],
            "bbox": [ref_bbox[0], ref_bbox[1], ref_bbox[2]-ref_bbox[0], ref_bbox[3]-ref_bbox[1]],
            "area": (ref_bbox[2]-ref_bbox[0])*(ref_bbox[3]-ref_bbox[1]), "iscrowd":0
        })
        ann_id += 1
        img_id += 1

    # write metadata.csv
    with open(meta_csv, "w", newline='') as f:
        writer = csv.DictWriter(f, fieldnames=list(metadata_rows[0].keys()))
        writer.writeheader()
        for r in metadata_rows:
            writer.writerow(r)

    with open(coco_json, "w") as f:
        json.dump(coco, f, indent=2)

    # zip files
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        for fn in os.listdir(img_dir):
            zf.write(os.path.join(img_dir, fn), arcname=os.path.join("images", fn))
        for fn in os.listdir(mask_dir):
            zf.write(os.path.join(mask_dir, fn), arcname=os.path.join("masks", fn))
        zf.write(meta_csv, arcname="metadata.csv")
        zf.write(coco_json, arcname="annotations_coco.json")

    print("Created dataset at:", zip_path)
    print("Images:", len(os.listdir(img_dir)))
    print("Masks:", len(os.listdir(mask_dir)))
    print("Metadata:", meta_csv)
    print("COCO annotations:", coco_json)

if __name__=="__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--out_dir", type=str, default="./synthetic_dataset")
    parser.add_argument("--num", type=int, default=200)
    args, unknown = parser.parse_known_args()
    main(args.out_dir, args.num)


Created dataset at: ./synthetic_dataset\synthetic_dataset.zip
Images: 200
Masks: 200
Metadata: ./synthetic_dataset\metadata.csv
COCO annotations: ./synthetic_dataset\annotations_coco.json


In [4]:
from google.colab import files
files.download('./synthetic_dataset/synthetic_dataset.zip')


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# baseline_eval.py
# Baseline evaluation for the synthetic dataset produced above.
# Usage: python baseline_eval.py --data_dir /path/to/synthetic_dataset

import os
import json
import csv
import math
import argparse
from PIL import Image
import numpy as np

def bbox_from_mask_np(m):
    ys, xs = np.where(m)
    if len(xs)==0:
        return None
    x0,y0 = int(xs.min()), int(ys.min())
    x1,y1 = int(xs.max()), int(ys.max())
    return [x0,y0,x1,y1]

def main(data_dir):
    meta_csv = os.path.join(data_dir, "metadata.csv")
    with open(meta_csv) as f:
        reader = csv.DictReader(f)
        rows = [r for r in reader]
    errors_w = []
    errors_h = []
    vol_rel_errors = []
    for r in rows:
        img_name = r["image_id"]
        mask_path = os.path.join(data_dir, "masks", img_name.replace("img_","mask_"))
        if not os.path.exists(mask_path):
            continue
        m = Image.open(mask_path).convert("L")
        m_np = np.array(m)
        ref_mask = (m_np==128).astype(np.uint8)
        obj_mask = (m_np==255).astype(np.uint8)
        ref_bbox = bbox_from_mask_np(ref_mask)
        obj_bbox = bbox_from_mask_np(obj_mask)
        if ref_bbox is None or obj_bbox is None:
            continue
        ref_pix_w = max(1, ref_bbox[2]-ref_bbox[0])
        ref_w_cm = 8.56
        est_ppcm = ref_pix_w / ref_w_cm
        proj_w_cm = (obj_bbox[2]-obj_bbox[0]) / est_ppcm
        proj_h_cm = (obj_bbox[3]-obj_bbox[1]) / est_ppcm
        gt_dims = json.loads(r["real_dims"])
        if r["shape"] in ["box","packet"]:
            gt_proj_w = gt_dims["L"]
            gt_proj_h = gt_dims["H"]
            gt_vol = gt_dims["L"]*gt_dims["W"]*gt_dims["H"]
        elif r["shape"] in ["cylinder","can","bottle"]:
            gt_proj_w = gt_dims["D"]
            gt_proj_h = gt_dims["H"]
            r_cm = gt_dims["D"]/2.0
            gt_vol = math.pi * r_cm*r_cm * gt_dims["H"]
        elif r["shape"] == "sphere":
            gt_proj_w = gt_dims["D"]
            gt_proj_h = gt_dims["D"]
            r_cm = gt_dims["D"]/2.0
            gt_vol = 4.0/3.0 * math.pi * r_cm**3
        else:
            continue
        errors_w.append(abs(proj_w_cm - gt_proj_w))
        errors_h.append(abs(proj_h_cm - gt_proj_h))
        # volume estimate (simple)
        if r["shape"] in ["box","packet"]:
            est_vol = proj_w_cm * (proj_h_cm) * (gt_dims["W"])
        elif r["shape"] in ["cylinder","can","bottle"]:
            est_vol = math.pi * (proj_w_cm/2.0)**2 * proj_h_cm
        elif r["shape"]=="sphere":
            est_vol = 4.0/3.0 * math.pi * (proj_w_cm/2.0)**3
        else:
            est_vol = None
        if est_vol is not None and gt_vol is not None and gt_vol>0:
            vol_rel_errors.append(abs(est_vol - gt_vol)/gt_vol * 100.0)

    print("Samples evaluated:", len(errors_w))
    print("Projected Width MAE (cm):", float(np.mean(errors_w)) if len(errors_w)>0 else None)
    print("Projected Height MAE (cm):", float(np.mean(errors_h)) if len(errors_h)>0 else None)
    print("Volume Relative Error (%) mean:", float(np.mean(vol_rel_errors)) if len(vol_rel_errors)>0 else None)
    print("Volume Relative Error (%) median:", float(np.median(vol_rel_errors)) if len(vol_rel_errors)>0 else None)

if __name__=="__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_dir", type=str, default="./synthetic_dataset")
    args, unknown = parser.parse_known_args()
    main(args.data_dir)


Samples evaluated: 200
Projected Width MAE (cm): 0.04987288620713198
Projected Height MAE (cm): 0.06429598691496455
Volume Relative Error (%) mean: 1.2527968072748406
Volume Relative Error (%) median: 0.9240883727251552


The script successfully generated a synthetic dataset! It created 200 images, 200 masks, a metadata CSV, and COCO annotations. All these files have been compressed into a zip file named synthetic_dataset.zip located in the ./synthetic_dataset directory.

The baseline evaluation script ran successfully! It processed 200 samples and reported the following metrics:

Projected Width MAE (Mean Absolute Error): 0.0499 cm. This means, on average, the estimated width of the objects differs from the true width by about 0.05 cm.
Projected Height MAE (Mean Absolute Error): 0.0643 cm. Similarly, the estimated height differs from the true height by about 0.06 cm.
Volume Relative Error (%): The mean relative error in volume estimation is 1.25%, and the median is 0.92%. This indicates that, on average, the estimated volume is very close to the actual volume, with most errors being well below 1.5%.

In [ ]:
import pandas as pd
import os

data_dir = "./synthetic_dataset"
meta_csv_path = os.path.join(data_dir, "metadata.csv")

if os.path.exists(meta_csv_path):
    df = pd.read_csv(meta_csv_path)
    print("First 5 lines of metadata.csv:")
    display(df.head())
else:
    print(f"The file {meta_csv_path} does not exist. Please run the dataset generation script in cell v64lfnNHuN-6 first.")

First 5 lines of metadata.csv:


,image_id,shape,real_dims,proj_bbox_px,ref_bbox_px,ppcm_used_for_render,est_ppcm_from_ref,volume_cm3
0,img_0000.png,packet,"{""L"": 8.12, ""W"": 15.86, ""H"": 6.41}","[124, 366, 183, 412]","[552, 414, 614, 453]",7.267,7.243,825.500
1,img_0001.png,can,"{""D"": 4.25, ""H"": 9.59}","[47, 307, 80, 381]","[544, 410, 610, 451]",7.780,7.710,136.046
2,img_0002.png,packet,"{""L"": 19.33, ""W"": 10.71, ""H"": 10.09}","[465, 23, 620, 104]","[543, 408, 611, 451]",8.030,7.944,2088.875
3,img_0003.png,can,"{""D"": 6.72, ""H"": 10.64}","[364, 72, 443, 197]","[514, 393, 614, 456]",11.765,11.682,377.372
4,img_0004.png,box,"{""L"": 12.82, ""W"": 9.5, ""H"": 6.76}","[490, 294, 576, 339]","[556, 420, 613, 456]",6.739,6.659,823.300
